### Sphere parameters for Makecloud

In [1]:
import numpy as np
import math
from astropy import constants as const
from astropy import units as u
#from astropy.constants import si

In [2]:
densUnit = u.solMass/(u.pc)**3.

In [3]:
densUnit

Unit("solMass / pc3")

In [4]:
boxM = 5900*u.solMass #box mass
boxL = 4.6*u.pc #box edge lenght
boxV = boxL**3
boxDens = boxM/boxV
boxDens.decompose(u.cgs.bases)

<Quantity 4.10235077e-21 g / cm3>

In [5]:
sphereM = 5000*u.solMass #sphere mass
sphereR = 2*u.pc #sphere radius
sphereV = 4./3*math.pi*sphereR**3
sphereDens = sphereM/sphereV
sphereDens.decompose(u.cgs.bases)

<Quantity 1.00982399e-20 g / cm3>

In [6]:
sphereM = 5000/10.*u.solMass #sphere mass
sphereR = 2/(10**(1./3))*u.pc #sphere radius
sphereV = 4./3*math.pi*sphereR**3
sphereDens = sphereM/sphereV
sphereDens.decompose(u.cgs.bases)

<Quantity 1.00982399e-20 g / cm3>

In [7]:
sphereM

<Quantity 500. solMass>

In [8]:
sphereR

<Quantity 0.92831777 pc>

In [9]:
sphereDens.decompose(u.cgs.bases)/(const.m_p.to(u.g))

<Quantity 6037.37146532 1 / cm3>

In [10]:
(1.2e-19*u.g/u.cm**3).decompose(u.cgs.bases)/(const.m_p.to(u.g)) # Bate (2019) paper value

<Quantity 71743.64887868 1 / cm3>

**Calcualtion of sound speed:**

Input $\mu$ and temperature:

In [11]:
mu = 2.381; T = 10*u.Kelvin
# mu = 2.36; T = 10*u.Kelvin

In [12]:
const.k_B

<<class 'astropy.constants.codata2018.CODATA2018'> name='Boltzmann constant' value=1.380649e-23 uncertainty=0.0 unit='J / K' reference='CODATA 2018'>

In [13]:
const.m_p

<<class 'astropy.constants.codata2018.CODATA2018'> name='Proton mass' value=1.67262192369e-27 uncertainty=5.1e-37 unit='kg' reference='CODATA 2018'>

In [14]:
cs = (const.k_B*T/mu/const.m_p)**(0.5)

In [15]:
cs

<Quantity 186.1928721 J(1/2) / kg(1/2)>

In [16]:
soundSpeed = cs.decompose(u.cgs.bases); soundSpeed

<Quantity 18619.28721045 cm / s>

But we need to know the sound speed in the code unit when we input initial parameters for the Phantom code.

In [17]:
const.G

<<class 'astropy.constants.codata2018.CODATA2018'> name='Gravitational constant' value=6.6743e-11 uncertainty=1.5e-15 unit='m3 / (kg s2)' reference='CODATA 2018'>

In [18]:
const.G.unit

Unit("m3 / (kg s2)")

In [19]:
timeUnitSi = (u.m**3/const.G.unit/u.kg)**(0.5); timeUnitSi

Unit("s")

In [20]:
timeUnit = (const.pc**3/(const.G)/const.M_sun)**(0.5)

In [21]:
timeUnit

<Quantity 4.70511238e+14 s>

In [22]:
velocityUnit = u.pc/timeUnit; velocityUnit

<Quantity 2.12534775e-15 pc / s>

In [23]:
soundSpeedDimless = soundSpeed/velocityUnit; soundSpeedDimless

<Quantity 8.76058388e+18 cm / pc>

In [24]:
soundSpeedDimless.decompose(u.cgs.bases)

<Quantity 2.83911188>

Calculation of maximum density that can be resolved:

In [25]:
Rg = const.R*u.mol/u.gram; Rg

<Quantity 8.31446262 J / (g K)>

### Calculation of Jeans length:

In [26]:
# sphereM = 50*u.solMass #sphere mass

# sphereDens = sphereM/sphereV
# sphereDens.decompose(u.cgs.bases)

In [27]:
JeansL = (math.pi*soundSpeed**2/const.G/sphereDens)**(1/2.); JeansL

<Quantity 3.3070398e+08 cm kg(1/2) pc(3/2) / (m(3/2) solMass(1/2))>

In [28]:
JeansL.decompose(u.cgs.bases)

<Quantity 1.27119532e+18 cm>

In [29]:
JeansL.decompose(u.cgs.bases).to(u.parsec)

<Quantity 0.41196635 pc>

In [30]:
# function to calculate rho_crit (Bate & Burkert 1997)
def calcRhoCrit (T, mu, Ntot, Mtot):
    Nnei = 32
    rhoCrit = (3./4/math.pi) * (5*Rg*T/2/const.G/mu)**3 * (Ntot/(2*Nnei)/Mtot)**2
    return rhoCrit.decompose(u.cgs.bases)

# function to calculate rho_crit (Hubber et al. 2006)
def calcRhoCritHubber (T, mu, Ntot, Mtot):
    ss2 = (const.k_B*T/mu/const.m_p) # sound speed ^ 2
    Nnei = 32
    rhoCritHubber = (math.pi*ss2/const.G)**3 * (math.pi/(6*Nnei*(Mtot/Ntot)))**2
    return rhoCritHubber.decompose(u.cgs.bases)

In [31]:
calcRhoCrit (T = 10*u.Kelvin, mu = 2.381, Ntot = 600000, Mtot = 500*const.M_sun)

<Quantity 4.75039079e-17 g / cm3>

In [32]:
calcRhoCritHubber (T = 10*u.Kelvin, mu = 2.381, Ntot = 3500000, Mtot = 500*const.M_sun)

<Quantity 1.44175777e-14 g / cm3>

### Jeans length at critical density:

In [33]:
rhoCrit = calcRhoCrit (T = 10*u.Kelvin, mu = 2.381, Ntot = 3500000, Mtot = 500*const.M_sun); print(rhoCrit)
JeansLengthCrit = (math.pi*soundSpeed**2/const.G/rhoCrit)**(1/2.)
print(JeansLengthCrit.decompose(u.cgs.bases).to(u.pc))

1.616452421128322e-15 g / cm3
0.0010296819918969198 pc


In [34]:
const.m_p.to(u.g)

<Quantity 1.67262192e-24 g>

In [35]:
rhoCrit

<Quantity 1.61645242e-15 g / cm3>

In [36]:
rhoCrit/(const.m_p.to(u.g))

<Quantity 9.66418291e+08 1 / cm3>

In [37]:
## I adopt this value
n_crit = 1e-15*u.g/u.cm**3/(const.m_p.to(u.g)) ## n for rho_crit =  10^-18 (g/cm^3) <-----------------
print(n_crit)

597863740.6556784 1 / cm3


In [38]:
1e-11*u.g/u.cm**3/(const.m_p.to(u.g)) ## n for rho_crit =  10^-11 (g/cm^3)

<Quantity 5.97863741e+12 1 / cm3>

In [39]:
u.AU.to(u.pc)

4.848136811133344e-06

In [40]:
u.pc.to(u.AU)

206264.80624548031

In [41]:
# softening = 3.11e-5 # ~6.5 AU, minimum sink radius is 2.8 times that (~18 AU) in MakeCloud
3.11e-5*u.pc.to(u.AU)

6.414835474234438

In [42]:
# Gas softening in MakeCloud
2.0e-8*u.pc.to(u.AU)

0.004125296124909607

### Jeans length at critical density for sink creation:

In [43]:
rhoSink = 1e-15*u.gram/u.cm**3 
JeansLengthSink = (math.pi*soundSpeed**2/const.G/rhoSink)**(1/2.)
print(JeansLengthSink.decompose(u.cgs.bases).to(u.AU)) 
print(JeansLengthSink.decompose(u.cgs.bases).to(u.pc)) # I adpot this for softening radius <-----------------
print("sink radius", JeansLengthSink.decompose(u.cgs.bases).to(u.pc)*2.8)
print("sink radius", JeansLengthSink.decompose(u.cgs.bases).to(u.AU)*2.8)

270.0285682073956 AU
0.0013091354415839056 pc
sink radius 0.0036655792364349354 pc
sink radius 756.0799909807076 AU


### Critical density for Bate (2009, MNRAS, 397, 232–248) initial condition:

In [44]:
calcRhoCrit (T = 10*u.Kelvin, mu = 2.46, Ntot = 3500000, Mtot = 50*const.M_sun)

<Quantity 1.46566862e-13 g / cm3>

In [45]:
calcRhoCritHubber (T = 10*u.Kelvin, mu = 2.46, Ntot = 3500000, Mtot = 50*const.M_sun)

<Quantity 1.30726961e-12 g / cm3>

In [46]:
u.pc.to(u.AU)

206264.80624548031

### Minimum space between particles in the cylinder

In [47]:
r   = 0.1
h   = 1.5
N   = 300000
Vol = np.pi*r*r*h
rd  = (Vol/N)**(1/3.)
print(rd, "pc")

0.005395602646429833 pc


In [48]:
h/rd

278.00416344456374

So, the maximum wave number $k_{max}$ must not be grater than 278.

---
Compute $\cal{M}$ for Price papers:

In [81]:
cs = 184*(u.m/u.s)
cs

<Quantity 184. m / s>

In [82]:
G = const.G
G

<<class 'astropy.constants.codata2018.CODATA2018'> name='Gravitational constant' value=6.6743e-11 uncertainty=1.5e-15 unit='m3 / (kg s2)' reference='CODATA 2018'>

In [98]:
alpha = 2
R_sphere = 0.375/2*u.pc
M_sphere = 50*u.solMass
Mach  = np.sqrt(( alpha * G * M_sphere) / (5/3 * cs**2 * R_sphere))
Mach.decompose()

<Quantity 6.37584441>

In [118]:
MachList = [0.1, 1, 2.5, 6.4, 10]
for Mach in MachList:
    cs = soundSpeed.to(u.m/u.s)
    R_sphere = 0.375/2*u.pc
    M_sphere = 50*u.solMass
    alpha  = (5/3 * cs**2 * Mach**2 * R_sphere) / (G * M_sphere)
    print(Mach, alpha.decompose())

0.1 0.000503784766214089
1 0.05037847662140891
2.5 0.3148654788838056
6.4 2.0635024024129085
10 5.03784766214089
